## Reading three TE accelerometers with a labJack T7-Pro\
Uses 9 analog inputs (AINs) to read the data at 200 Hz.

Craig Lage - Sep 09, 2021

Relevant Documentation:
 
LJM Library:
    LJM Library Installer:
        https://labjack.com/support/software/installers/ljm
    LJM Users Guide:
        https://labjack.com/support/software/api/ljm
    Opening and Closing:
        https://labjack.com/support/software/api/ljm/function-reference/opening-and-closing
    Multiple Value Functions(such as eWriteNames):
        https://labjack.com/support/software/api/ljm/function-reference/multiple-value-functions
    Timing Functions(such as StartInterval):
        https://labjack.com/support/software/api/ljm/function-reference/timing-functions
 
T-Series and I/O:
    Modbus Map:
        https://labjack.com/support/software/api/modbus/modbus-map
    Analog Inputs:
        https://labjack.com/support/datasheets/t-series/ain

In [ ]:
import sys
import time, datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from labjack import ljm  # Needed pip install labjack-ljm

In [ ]:
dir(ljm)

In [ ]:
ljm.readLibraryConfigS('LJM_ETHERNET_SEND_RECEIVE_TIMEOUT_MS')

In [ ]:
ljm.writeLibraryConfigS('LJM_ETHERNET_SEND_RECEIVE_TIMEOUT_MS', 26000)

In [ ]:
# Open LabJack
handle = ljm.openS("T7", "wifi", "139.229.164.249")  

In [ ]:
# Open LabJack
handle = ljm.openS("T7", "ethernet", "139.229.164.248")  

In [ ]:
info = ljm.getHandleInfo(handle)
print("Opened a LabJack with Device type: %i, Connection type: %i,\n"
      "Serial number: %i, IP address: %s, Port: %i,\nMax bytes per MB: %i" %
      (info[0], info[1], info[2], ljm.numberToIP(info[3]), info[4], info[5]))

deviceType = info[0]
print(deviceType)

In [ ]:
# Open LabJack
handle = ljm.openS("T7", "wifi", "139.229.164.249")  

In [ ]:
# Open LabJack
handle = ljm.openS("T7", "wifi", "139.229.164.249")  

In [ ]:
# Structures to hold the calibration data and connection information
class zeroOffset:
    def __init__(self, off_x, off_y, off_z):
        # This is the reading in Volts when accel = 0
        self.x = off_x
        self.y = off_y
        self.z = off_z
        
class gMult:
    def __init__(self, mult_x, mult_y, mult_z):
        # This is the conversion to acceleration in V/g
        self.x = mult_x
        self.y = mult_y
        self.z = mult_z
        
class AIN_name:
    def __init__(self, ain_x, ain_y, ain_z):
        # This is where the sensors are connected to the labJack
        self.x = ain_x
        self.y = ain_y
        self.z = ain_z

class calData:
    def __init__(self, serial="", ain_x="", ain_y="", ain_z="", off_x=0.0, off_y=0.0, off_z=0.0, mult_x=1.0, mult_y=1.0, mult_z=1.0):
        # The serial number is imprinted on the accelerometer
        self.serial = serial
        self.AIN_name = AIN_name(ain_x, ain_y, ain_z)
        self.zeroOffset = zeroOffset(off_x, off_y, off_z)
        self.gMult = gMult(mult_x, mult_y, mult_z)
        
calDict = {}
calDict["1"] = calData(serial="A395429", ain_x="AIN1", ain_y="AIN2", ain_z="AIN3", off_x=2.49017, off_y=2.44424, off_z=2.44589, mult_x=0.98959, mult_y=0.98572, mult_z=0.99946)
calDict["2"] = calData(serial="A395423", ain_x="AIN4", ain_y="AIN5", ain_z="AIN6", off_x=2.49874, off_y=2.49595, off_z=2.41423, mult_x=0.99740, mult_y=1.00142, mult_z=0.99595)
calDict["3"] = calData(serial="A395446", ain_x="AIN7", ain_y="AIN8", ain_z="AIN9", off_x=2.47830, off_y=2.48088, off_z=2.41385, mult_x=0.97957, mult_y=0.98699, mult_z=1.00376)

In [ ]:
# Create list of AIN names and initialize

# Ensure triggered stream is disabled.
ljm.eWriteName(handle, "STREAM_TRIGGER_INDEX", 0)
# Enabling internally-clocked stream.
ljm.eWriteName(handle, "STREAM_CLOCK_SOURCE", 0)

# All negative channels are single-ended, AIN0 and AIN1 ranges are
# +/-10 V, stream settling is 0 (default) and stream resolution index
# is 0 (default).

aRange = 10.0 # +/- 10.0 Volts
aSettle = 0 # 0 microsecond settling time
resIndex = 0
aNames = ["AIN_ALL_NEGATIVE_CH", "STREAM_SETTLING_US", "STREAM_RESOLUTION_INDEX"] # List of set-up parameters
aValues = [ljm.constants.GND, aSettle, resIndex] # List of set-up values

aScanListNames = [] # List of AIN names which will be read
offsets = []
gMults = []
for name in ["1", "2", "3"]:
    for axis in ["x", "y", "z"]:
        exec(f"aName = calDict['{name}'].AIN_name.{axis}")
        aScanListNames.append(aName)
        aNames.append(aName+"_RANGE")
        aValues.append(aRange)
        exec(f"off = calDict['{name}'].zeroOffset.{axis}")
        offsets.append(off)
        exec(f"gMult = calDict['{name}'].gMult.{axis}")
        gMults.append(gMult)

offsets = np.array(offsets)
gMults = np.array(gMults)
         
# Write the analog inputs' negative channels (when applicable), ranges,
# stream settling time and stream resolution configuration.
numFrames = len(aNames)
ljm.eWriteNames(handle, numFrames, aNames, aValues)


In [ ]:
def readStream(aScanListNames, handle=handle, scanRate=200, readTime=1.0, offsets=offsets, gMults=gMults):
    # This reads the accelerometers for a time readTime
    # and returns a Pandas timeSeries with the data
    # aScanListNames is the list of AIN ports (9 in total)
    # handle is the handle for talking to the labJack
    # scanRate is the read frequency in Hertz
    # readTime is the total time of read in seconds
    # calDict is the dictionary with the calibration data
    # The function returns a Pandas dataframe with the three 
    # accelerometers times three axes results
    
    numAddresses = len(aScanListNames)
    aScanList = ljm.namesToAddresses(numAddresses, aScanListNames)[0]
    scansPerRead = int(scanRate * readTime)
    try:
        # Configure and start stream
        scanRate = ljm.eStreamStart(handle, scansPerRead, numAddresses, aScanList, scanRate)
        start = datetime.datetime.now()
        # Stream the data
        ret = ljm.eStreamRead(handle)
        # Stop the stream
        ljm.eStreamStop(handle)
        aData = ret[0]
        # Reshape the data
        newData = np.resize(aData, (scansPerRead, numAddresses))
        # Convert to g
        accelData = (newData - offsets) / gMults
        # Create the timestamps
        end = start + datetime.timedelta(seconds = readTime)
        date_rng = pd.date_range(start=start, end=end, periods=scansPerRead)
        # Create the Pandas dataFrame
        df = pd.DataFrame(accelData, index=date_rng, 
                          columns=['ELM2', 'AZM2', 'ZM2', 'ELT', 'ZT', 'AZT', 'ELM1', 'AZM1', 'ZM1'])
    except ljm.LJMError:
        ljme = sys.exc_info()[1]
        print(ljme)
    except Exception:
        e = sys.exc_info()[1]
        print(e)
    # Return the dataframe
    return df

In [ ]:
time.sleep(5.0)
df = readStream(aScanListNames, handle=handle, scanRate=200, readTime=5.0, offsets=offsets, gMults=gMults)

In [ ]:
df.head(5)

In [ ]:
# Plot the data

sub_df = df[df.index[320] : df.index[500]]

plt.figure(figsize=(8,8))
plt.subplots_adjust(hspace=0.5, wspace=0.5)
plt.suptitle("Post-Installation Test 24Feb22", fontsize=18)
plt.subplot(3,3,1)
ax1A = sub_df['ELM2'].plot(color='red')
ax1A.set_title("Elevation M2", fontsize=16)
ax1A.set_ylabel("Acceleration(g)")
plt.subplot(3,3,2)
ax1A = sub_df['ELT'].plot(color='red')
ax1A.set_title("Elevation Truss", fontsize=16)
ax1A.set_ylabel("Acceleration(g)")
plt.subplot(3,3,3)
ax1A = sub_df['ELM1'].plot(color='red')
ax1A.set_title("Elevation M1", fontsize=16)
ax1A.set_ylabel("Acceleration(g)")
plt.subplot(3,3,4)
ax1A = sub_df['AZM2'].plot(color='blue')
ax1A.set_title("Azimuth M2", fontsize=16)
ax1A.set_ylabel("Acceleration(g)")
plt.subplot(3,3,5)
ax1A = sub_df['AZT'].plot(color='blue')
ax1A.set_title("Azimuth Truss", fontsize=16)
ax1A.set_ylabel("Acceleration(g)")
plt.subplot(3,3,6)
ax1A = sub_df['AZM1'].plot(color='blue')
ax1A.set_title("Azimuth M1", fontsize=16)
ax1A.set_ylabel("Acceleration(g)")
plt.subplot(3,3,7)
ax1A = sub_df['ZM2'].plot(color='green')
ax1A.set_title("Optical-Axis M2", fontsize=16)
ax1A.set_ylabel("Acceleration(g)")
plt.subplot(3,3,8)
ax1A = sub_df['ZT'].plot(color='green')
ax1A.set_title("Optical-Axis Truss", fontsize=16)
ax1A.set_ylabel("Acceleration(g)")
plt.subplot(3,3,9)
ax1A = sub_df['ZM1'].plot(color='green')
ax1A.set_title("Optical-Axis M1", fontsize=16)
ax1A.set_ylabel("Acceleration(g)")

plt.savefig("/Users/cslage/Research/LSST/code/labJack/accel_data/Summit_Install_Test_24Feb22.pdf")

In [ ]:
# Close handles
ljm.close(handle)

The commands below are useful if the device gets "locked up".

In [ ]:
ljm.closeAll()

In [ ]:
ljm.eStreamStop(handle)